In [1]:
import pandas as pd

df = pd.read_csv('../../data/insiders_downloaded.csv')
df.info()
descriptions = df['Description']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61397 entries, 0 to 61396
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   61396 non-null  float64
 1   id           61397 non-null  object 
 2   zip          61397 non-null  object 
 3   file name    61397 non-null  object 
 4   entry        61397 non-null  object 
 5   Description  61397 non-null  object 
 6   CPV          61397 non-null  object 
 7   URI          61397 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.7+ MB


## Data balancing

In [2]:
# Calculate the length of each description
description_lengths = descriptions.apply(len)

Q1 = description_lengths.quantile(0.25)
Q3 = description_lengths.quantile(0.75)
IQR = Q3 - Q1

# Determine outlier thresholds
lower_threshold = Q1 - 1.5 * IQR
upper_threshold = Q3 + 1.5 * IQR

print(lower_threshold,upper_threshold)

75.5 471.5


In [3]:
# Filter descriptions within thresholds
filtered_df = df[(description_lengths >= lower_threshold) & (description_lengths <= upper_threshold)]
filtered_df = filtered_df.reset_index(drop=True)
print(filtered_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57048 entries, 0 to 57047
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   57048 non-null  float64
 1   id           57048 non-null  object 
 2   zip          57048 non-null  object 
 3   file name    57048 non-null  object 
 4   entry        57048 non-null  object 
 5   Description  57048 non-null  object 
 6   CPV          57048 non-null  object 
 7   URI          57048 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.5+ MB
None


## Text preprocessing

In [4]:
import nltk
from nltk.corpus import stopwords
import spacy

# Load spaCy's Spanish language model
nlp = spacy.load('es_core_news_sm')

# Download NLTK stop words
nltk.download('stopwords')

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Define Spanish stop words
spanish_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re
import spacy

nlp = spacy.load('es_core_news_sm')

def preprocess_text(text):
    # Tokenization
    tokens = nltk.word_tokenize(text, language='spanish')
    
    # Remove unnecessary data
    tokens = [re.sub(r'\b\d{2,}\b|(?:\d{1,2}[\/\-\.]){2,}\d{2,4}|\b(?:tel|email)\b', '', token) for token in tokens]
    tokens = [token for token in tokens if token.lower() not in ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]
    
    # Remove non-alphabetic data
    tokens = [re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]', '', token) for token in tokens]
    
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    
    # Remove stop words
    tokens = [token for token in tokens if token not in spanish_stopwords]
    
    # Lemmatization
    doc = nlp(' '.join(tokens))
    tokens = [token.lemma_ for token in doc]
    
    return ' '.join(tokens)

In [8]:
filtered_descriptions = filtered_df['Description']

preprocessed_descriptions = [preprocess_text(desc) for desc in filtered_descriptions]
filtered_df['processed_description'] = preprocessed_descriptions

Add new column for non preprocessed models (just lowercasing because some models are case sensitive)

In [9]:
filtered_df['lowercase_description'] = filtered_descriptions.str.lower()

In [10]:
# drop unnecessary columns
filtered_df = filtered_df.drop(columns=['Unnamed: 0', 'id'])
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57048 entries, 0 to 57047
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   zip                    57048 non-null  object
 1   file name              57048 non-null  object
 2   entry                  57048 non-null  object
 3   Description            57048 non-null  object
 4   CPV                    57048 non-null  object
 5   URI                    57048 non-null  object
 6   processed_description  57048 non-null  object
 7   lowercase_description  57048 non-null  object
dtypes: object(8)
memory usage: 3.5+ MB


In [11]:
# Save the DataFrame to a CSV file
filtered_df.to_csv('../../data/insiders_with_preprocessed_descriptions.csv')